In [5]:
import os
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]


In [4]:
import opendatasets as od
dataset = "https://www.kaggle.com/competitions/llm-detect-ai-generated-text"

In [3]:
od.download(dataset)

Skipping, found downloaded files in ".\llm-detect-ai-generated-text" (use force=True to force download)


In [4]:
import pandas as pd
import numpy as np

In [5]:
current_dir = os.getcwd()
df_train = pd.read_csv(f'{current_dir}/llm-detect-ai-generated-text/train_essays.csv')

In [6]:
df_train.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [7]:
df_test = pd.read_csv(f'{current_dir}/llm-detect-ai-generated-text/test_essays.csv')
df_test.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [8]:
df_prompt = pd.read_csv(f'{current_dir}/llm-detect-ai-generated-text/train_prompts.csv')
df_prompt.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [9]:
df_train.text.str.len().describe()

count    1378.000000
mean     3169.050798
std       920.588198
min      1356.000000
25%      2554.250000
50%      2985.500000
75%      3623.750000
max      8436.000000
Name: text, dtype: float64

In [10]:
df_train.shape

(1378, 4)

In [11]:
# Merge train_essays with train_prompts to get prompt information
train_data = pd.merge(df_train, df_prompt, on='prompt_id', how='left')
train_data.sample(10)

,id,prompt_id,text,generated,prompt_name,instructions,source_text
830,961d4ea9,0,A decline in the use of motor vehicles would g...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1252,ea641ffa,0,The laws have changed. We now have days where ...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1197,dea147b0,0,"""All of our development since World War II has...",0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
825,94e15d84,0,Everyone assumes that the best way of transpor...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1359,fc66a299,0,There are many advantages today of limiting ca...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
75,115bceb1,1,"In the westminster system, the head of state a...",0,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...
259,341bd6dd,1,"Dear State Senate, The Electoral College is in...",0,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...
1156,d6164eb0,0,Ever wondered how the streets of a very popula...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
659,79f714f8,0,Personal automobiles and cars have played a ma...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
509,633b4981,0,Why is is beneficial to have limited car usage...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."


In [12]:
# Handling missing values
train_data.dropna(subset=['text'], inplace=True)
df_test.dropna(subset=['text'], inplace=True)

In [55]:
# Clean the text data (remove special characters, lowercase, remove stopwords, etc.)
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Join tokens back into text
    cleaned_text = ' '.join(filtered_tokens)
    
    return cleaned_text

In [56]:
train_data['cleaned_text'] = train_data['text'].apply(clean_text)
df_test['cleaned_text'] = df_test['text'].apply(clean_text)

In [57]:
df_test.head()

,id,prompt_id,text,cleaned_text
0,0000aaaa,2,Aaa bbb ccc.,aaa bbb ccc
1,1111bbbb,3,Bbb ccc ddd.,bbb ccc ddd
2,2222cccc,4,CCC ddd eee.,ccc ddd eee


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_data['generated'] = label_encoder.fit_transform(train_data['generated'])
train_data

,id,prompt_id,text,generated,prompt_name,instructions,source_text,cleaned_text
0,0059830c,0,Cars. Cars have been around since they became ...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",cars cars around since became famous henry for...
1,005db917,0,Transportation is a large necessity in most co...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",transportation large necessity countries world...
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",americas love affair vehicles seems cooling sa...
3,00940276,0,How often do you ride in a car? Do you drive a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",often ride car drive one motor vehicle work st...
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",cars wonderful thing perhaps one worlds greate...
...,...,...,...,...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...,fuss elector college many people get confused ...
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",limiting car usage many advantages putting lot...
1375,ffa247e0,0,There's a new trend that has been developing f...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",theres new trend developing years soon full th...
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",know cars big part society today however cars ...


In [59]:
X = train_data['cleaned_text']
y = train_data['generated']


In [60]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_val

597     0
700     0
1222    0
1145    0
602     0
       ..
506     0
615     0
365     0
828     0
1281    0
Name: generated, Length: 276, dtype: int64

In [62]:
from transformers import BertTokenizer, TFBertForSequenceClassification


# Load BERT tokenizer and encode the text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [74]:
# train_encodings = [tokenizer(text, truncation=True, padding='max_length', max_length=512)["input_ids"] for text in X_train]
# val_encodings = [tokenizer(text, truncation=True, padding='max_length', max_length=512)["input_ids"] for text in X_val]

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors="tf")
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, return_tensors="tf")


In [75]:
import tensorflow as tf

# train_dataset = np.array(train_encodings, dtype='int64')
# val_dataset = np.array(val_encodings, dtype='int64')
# y_train =  np.array(y_train, dtype='int64')
# y_val =  np.array(y_val, dtype='int64')

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.astype(np.int32)
)).shuffle(len(X_train)).batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val.astype(np.int32)
)).batch(8)

val_dataset

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [79]:
# Load pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
# Define optimizer, loss, and compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

Model: "tf_bert_for_sequence_classification_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_265 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
model.fit(train_dataset, epochs=3,batch_size=16, validation_data=val_dataset)

Epoch 1/3
 21/138 [===>..........................] - ETA: 2:28:07 - loss: 0.0976 - accuracy: 0.9881

In [ ]:
results = model.evaluate(val_dataset)
print(f'Validation Loss: {results[0]}, Validation Accuracy: {results[1]}')

In [ ]:
df_test.dropna(subset=['text'], inplace=True)
df_test['cleaned_text'] = df_test['text'].apply(clean_text)

In [ ]:
# Tokenize the cleaned text data using the BERT tokenizer
test_encodings = tokenizer(df_test['cleaned_text'].tolist(), truncation=True, padding=True)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings)
)).batch(8)

In [ ]:
# Make predictions on the test set
predictions = model.predict(test_dataset)

In [ ]:
# Assuming your model predicts probabilities for binary classification
predicted_probabilities = predictions.logits

In [ ]:
# Convert probabilities to class labels (0 or 1)
threshold = 0.5  # Define a threshold for classification
predicted_labels = (predicted_probabilities > threshold).astype(int)

In [ ]:
# Assuming you want to create a DataFrame with essay IDs and predicted labels
essay_ids = df_test['id'].tolist()
results = pd.DataFrame({'id': essay_ids, 'predicted_label': predicted_labels.flatten()})


In [ ]:
# Optionally, you can save the results to a CSV file
results.to_csv('predicted_results.csv', index=False)